In [1]:
from mne.io import concatenate_raws, read_raw_edf
import mne
import glob
import numpy as np
import xlrd
import pandas as pd

In [2]:
path = "../SE_parameter/刺激参数（第四批次）.xlsx"
df = pd.read_excel(path,engine='openpyxl')
#df= df.astype({'完成时间':'object'})
#对df进行排序，为了和glob.glob的顺序相匹配
df

,姓名,性别,年龄（岁）,治疗时间,刺激方案,标签
0,成思思,NaN,NaN,20220808,1,1
1,成思思,女,20.0,20220803,2,1
2,成思思,NaN,NaN,20220810,3,1
3,成思思,NaN,NaN,20220805,4,1
4,董沁鑫,NaN,NaN,20220720,1,1
...,...,...,...,...,...,...
91,张海峰,男,46.0,20220819,4,0
92,张会,女,52.0,20220713,1,1
93,张会,NaN,NaN,20220718,2,0
94,张会,NaN,NaN,20220720,3,1


In [3]:
df=df.sort_values(by=["姓名","治疗时间"])
df

,姓名,性别,年龄（岁）,治疗时间,刺激方案,标签
83,严聪慧,男,17.0,20220803,4,0
82,严聪慧,NaN,NaN,20220805,3,0
80,严聪慧,NaN,NaN,20220808,1,1
81,严聪慧,NaN,NaN,20220810,2,1
11,付雨欣(付宇欣),女,15.0,20220715,4,1
...,...,...,...,...,...,...
5,董沁鑫,NaN,NaN,20220725,2,1
73,谢思沂,女,26.0,20220713,2,1
75,谢思沂,NaN,NaN,20220715,4,1
74,谢思沂,NaN,NaN,20220718,3,1


In [4]:
df.drop(df.loc[df['标签']=='-'].index, inplace=True)
df=df.reset_index()
df=df.drop(['index'],axis=1)
df

,姓名,性别,年龄（岁）,治疗时间,刺激方案,标签
0,严聪慧,男,17.0,20220803,4,0
1,严聪慧,NaN,NaN,20220805,3,0
2,严聪慧,NaN,NaN,20220808,1,1
3,严聪慧,NaN,NaN,20220810,2,1
4,付雨欣(付宇欣),女,15.0,20220715,4,1
...,...,...,...,...,...,...
90,董沁鑫,NaN,NaN,20220725,2,1
91,谢思沂,女,26.0,20220713,2,1
92,谢思沂,NaN,NaN,20220715,4,1
93,谢思沂,NaN,NaN,20220718,3,1


In [5]:
df.shape

(95, 6)

In [6]:
ECT_code = []

In [7]:
for i in range(df.shape[0]):
    ECT_code.append(df.iloc[i, 4])

In [8]:
ECT_code = np.array(ECT_code)

In [9]:
ECT_code

array([4, 3, 1, 2, 4, 1, 2, 3, 3, 1, 4, 2, 4, 2, 3, 1, 1, 2, 3, 4, 4, 1,
       2, 3, 2, 3, 4, 1, 2, 1, 3, 4, 2, 1, 3, 1, 2, 4, 3, 1, 4, 2, 3, 4,
       3, 2, 1, 1, 4, 3, 2, 1, 3, 2, 4, 2, 4, 1, 3, 3, 1, 2, 4, 2, 3, 1,
       4, 1, 3, 4, 2, 4, 1, 3, 2, 3, 4, 1, 2, 3, 2, 1, 4, 3, 4, 2, 1, 3,
       4, 1, 2, 2, 4, 3, 1], dtype=int64)

In [10]:
# rest_path = r"../patient_data/静息脑电（第一批次）/"
therapy_path=r"../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考/"

# rest_subdirectory = sorted(glob.glob(rest_path+"*"))
# therapy_subdirectory = sorted(glob.glob(therapy_path+"*"))
therapy_file_path = sorted(glob.glob(therapy_path+"*"))

In [11]:
len(therapy_file_path)

95

In [12]:
path_parameter_dict={}
path_result_dict={}

In [13]:
#构建一个 {路径:ECT参数} 的字典，方便打乱顺序后，仍能保持一一对应
for i in range(len(therapy_file_path)):
    path_parameter_dict[therapy_file_path[i]]=ECT_code[i]
    path_result_dict[therapy_file_path[i]]=df.iloc[i,5]

In [14]:
path_parameter_dict

{'../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-严聪慧-20220803.edf': 4,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-严聪慧-20220805.edf': 3,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-严聪慧-20220808.edf': 1,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-严聪慧-20220810.edf': 2,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-付宇欣-20220715.edf': 4,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-付宇欣-20220718.edf': 1,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-付宇欣-20220720.edf': 2,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-付宇欣-20220722.edf': 3,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-何晓丽-20220824.edf': 3,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-何晓丽-20220826.edf': 1,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-何晓丽-20220829.edf': 4,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-何晓丽-20220831.edf': 2,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-倪泽菁-20220826.edf': 4,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-倪泽菁-20220829.edf': 2,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-倪泽菁-20220831.edf': 3,
 '../patie

In [15]:
path_result_dict

{'../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-严聪慧-20220803.edf': 0,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-严聪慧-20220805.edf': 0,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-严聪慧-20220808.edf': 1,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-严聪慧-20220810.edf': 1,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-付宇欣-20220715.edf': 1,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-付宇欣-20220718.edf': 1,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-付宇欣-20220720.edf': 1,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-付宇欣-20220722.edf': 1,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-何晓丽-20220824.edf': 1,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-何晓丽-20220826.edf': 1,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-何晓丽-20220829.edf': 1,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-何晓丽-20220831.edf': 1,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-倪泽菁-20220826.edf': 1,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-倪泽菁-20220829.edf': 1,
 '../patient_data/治疗脑电-导出/第四批次治疗脑电/平均参考\\AV-倪泽菁-20220831.edf': 1,
 '../patie

In [16]:
#观察文件名，发现所有文件名均为“-人名-”，所以取人名前后对照，如果不同则表示是不同的人
#思路：首先将所有人名取出，然后随机打乱顺序，前20个人作为训练数据，后4个人作为测试数据
therapy_file_path[0].replace('\\','/').split('/')[-1].split('-')[1]

'严聪慧'

In [17]:
#获得名单
def getNameList(file_path,name_list):
    last_name='1'
    for i in file_path:
        cur_name=i.replace('\\','/').split('/')[-1].split('-')[1]
        if (last_name!=cur_name):
            name_list.append(cur_name)
        last_name=cur_name      

In [18]:
#打乱名单，体现随机性
np.random.seed(42)
name_list=[]
getNameList(therapy_file_path,name_list)
np.random.shuffle(name_list)
name_list

['夏帆',
 '王云芝',
 '严聪慧',
 '王诗雯',
 '张海峰',
 '尤李心怡',
 '徐洋忠',
 '付宇欣',
 '胡杨',
 '吴华丽',
 '何晓丽',
 '彭国芬',
 '李萌',
 '倪泽菁',
 '刘锦珍',
 '董沁鑫',
 '王敏敏',
 '肖敏霞',
 '谢思沂',
 '唐心诚',
 '张会',
 '成思思',
 '罗文彤',
 '吴妍']

In [19]:
len(name_list)

24

In [20]:
#此处设定有多少患者用于训练集(人数)
train_patient_num=20
total_patient_num=len(name_list)
train_therapy_file_path=[]
test_therapy_file_path=[]
for i in range(train_patient_num):
    temp_file_path=sorted(glob.glob(therapy_path+'*'+name_list[i]+'*'))
    for j in temp_file_path:
        train_therapy_file_path.append(j)
for i in range(train_patient_num,total_patient_num):
    temp_file_path=sorted(glob.glob(therapy_path+'*'+name_list[i]+'*'))
    for j in temp_file_path:
        test_therapy_file_path.append(j)

In [21]:
#这里必须要排序
train_therapy_file_path.sort()
test_therapy_file_path.sort()

In [22]:
#统计未发作和发作的样本数
trigger_num=0
non_trigger_num=0
for i in range(df.shape[0]):
    # 找不到未发作，即发作
    if df.iloc[i, 5]==1:
        trigger_num+=1
    else:
        non_trigger_num+=1
print(trigger_num/non_trigger_num)

multiple=trigger_num/non_trigger_num

5.333333333333333


In [23]:
multiple = round(multiple)
multiple

5

In [24]:
non_trigger_num

15

In [25]:
trigger_num+non_trigger_num*multiple

155

In [26]:
#抖动
def jitter(data):
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            #抖动因子
            jitter_factor=np.random.randint(1,11)
            #抖动方向
            jitter_direction=np.random.randint(1,3)
            #变化在 *0.9 至 *1.1内
            if jitter_direction==1:
                data[i][j]= data[i][j]*(1+jitter_factor/100)
            else:
                data[i][j]= data[i][j]*(1-jitter_factor/100)

In [27]:
#翻转
def flip(data):
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            data[i][j]=-data[i][j]

In [28]:
#随机通道翻转
def random_flip(data):
    flip_channel=np.random.randint(0,15)
    for i in range (data.shape[1]):
        data[flip_channel][i]=-data[flip_channel][i]

In [29]:
#扩大/压缩
def scale_up(data):
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            data[i][j]= data[i][j]*1.1
def scale_down(data):
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            data[i][j]= data[i][j]*0.9

In [30]:
# #排列
# def permutation(data,p_strategy):
#     for i in range(0,16,2):
#         #把2s片段分为4组,假设序号为1,2,3,4
#         p1_start=int(i*1024)
#         p1_end=int((i+0.5)*1024)
#         p2_start=int((i+0.5)*1024)
#         p2_end=int((i+1)*1024)
#         p3_start=int((i+1)*1024)
#         p3_end=int((i+1.5)*1024)
#         p4_start=int((i+1.5)*1024)
#         p4_end=int((i+2)*1024)
#         for j in range(15):
#             #排列方案为1时， 1,2,3,4-->2,3,4,1
#             if p_strategy==1: 
#                 data_temp=[]
#                 data_temp.append(data[j][p2_start:p2_end])
#                 data_temp.append(data[j][p3_start:p3_end])
#                 data_temp.append(data[j][p4_start:p4_end])
#                 data_temp.append(data[j][p1_start:p1_end])
#                 data_temp=np.array(data_temp)
#                 data_temp=data_temp.reshape(1,-1)
#                 data_temp=data_temp[0]
#                 data=np.array(data)
#                 data[j,p1_start:p4_end]=data_temp
#             #排列方案为2时， 1,2,3,4-->3,4,1,2
#             elif p_strategy==2:
#                 data_temp=[]
#                 data_temp.append(data[j][p3_start:p3_end])
#                 data_temp.append(data[j][p4_start:p4_end])
#                 data_temp.append(data[j][p1_start:p1_end])
#                 data_temp.append(data[j][p2_start:p2_end])
#                 data_temp=np.array(data_temp)
#                 data_temp=data_temp.reshape(1,-1)
#                 data_temp=data_temp[0]
#                 data[j,p1_start:p4_end]=data_temp
#             #排列方案为3时， 1,2,3,4-->4,1,2,3
#             elif p_strategy==3:
#                 data_temp=[]
#                 data_temp.append(data[j][p4_start:p4_end])
#                 data_temp.append(data[j][p1_start:p1_end])
#                 data_temp.append(data[j][p2_start:p2_end])
#                 data_temp.append(data[j][p3_start:p3_end])
#                 data_temp=np.array(data_temp)
#                 data_temp=data_temp.reshape(1,-1)
#                 data_temp=data_temp[0]
#                 data[j,p1_start:p4_end]=data_temp

In [31]:
#检测EEG文件是否有flag标记,返回true/false
def judgeflag(file_path,flag):
    raw = read_raw_edf(file_path,preload=False)
    events_from_annot, event_dict = mne.events_from_annotations(raw)
    if flag in event_dict:
        return True
    else:
        return False

In [32]:
# raw=read_raw_edf(train_therapy_file_path[0],preload=False)

In [33]:
# events_from_annot, event_dict = mne.events_from_annotations(raw)

In [34]:
# event_dict

In [35]:
# events_from_annot

In [36]:
# data,times=raw[:,0:1024*16]
# raw2=mne.io.RawArray(data,raw.info)

In [37]:
# 数据采样率1024Hz,文件路径；采集时间；是否为测试集标志;数据增强策略;未发作的第几次随机通道翻转
def get_se_epochs(path,select_time,is_test,DA_strategy,random_flip_no):
    #选择15个通道
    chs1 = ['EEG Fp1-AV', 'EEG Fp2-AV', 'EEG F3-AV', 'EEG F4-AV', 'EEG C3-AV', 'EEG C4-AV', 'EEG P3-AV', 'EEG P4-AV',
       'EEG O1-AV', 'EEG O2-AV', 'EEG T3-AV', 'EEG T4-AV', 'EEG T5-AV', 'EEG T6-AV','EEG Cz-AV']
#     chs2 =['EEG Fp1-A1', 'EEG Fp2-A2', 'EEG F3-A1','EEG F4-A2','EEG C3-A1','EEG C4-A2','EEG P3-A1','EEG P4-A2','EEG O1-A1',
#            'EEG O2-A2','EEG T3-A1','EEG T4-A2','EEG T5-A1','EEG T6-A2','EEG Cz-AAV']
    # select_time 采集的时间
    raw=read_raw_edf(path,preload=False)
    print(raw.info.ch_names)
    #第一批数据采样为1024Hz，第二批变成了512Hz
#     if chs1[0] in raw.info.ch_names:
#         chs=chs1  
#         freq=1024
#     else:
#         chs=chs2
#         freq=512
    chs=chs1
    freq=512
    raw=raw.pick_channels(chs)
    events_from_annot, event_dict = mne.events_from_annotations(raw)
    # 找到SE，麻醉标记,save函数的参数单位是s，所以要除以1024
    start_time=events_from_annot[event_dict['SE']-2][0]
    end_time=start_time+select_time*freq
    print(start_time,end_time)
    #选择所有的15通道
    data,times = raw[:,start_time:end_time]
    print(times)
    #不做变化
    if (DA_strategy==0): 
        path_name_suffix='(original)'
    #抖动
    elif (DA_strategy==1):
        jitter(data)
        path_name_suffix='(jitter)'
    #翻转
    elif (DA_strategy==2):
        flip(data)
        path_name_suffix='(flip)'
    #扩大
    elif (DA_strategy==3):
        scale_up(data)
        path_name_suffix='(scale_up)'  
    #缩小
    elif (DA_strategy==4):
        scale_down(data)
        path_name_suffix='(scale_down)'  
    #排列1
    elif (DA_strategy==5):
        permutation(data,1)
        path_name_suffix='(permutation_1)'  
    #排列2
    elif (DA_strategy==6):
        permutation(data,2)
        path_name_suffix='(permutation_2)'  
    #排列3
    elif (DA_strategy==7):
        permutation(data,3)
        path_name_suffix='(permutation_3)'  
    #随机通道翻转
    elif (DA_strategy==8):
        random_flip(data)
        path_name_suffix='random_flip_'+str(random_flip_no)
    raw_temp=mne.io.RawArray(data,raw.info)
    path_name=path.split('\\')[1:]
    path_name="".join(path_name)
    path_name=path_name.split('.')[0]
    #在这里将训练集和测试集分开,需要新建SE_train和SE_test两个文件夹
    if (is_test==False):
        path_name=r"../SE_train/" + path_name +path_name_suffix+".fif"
    else:   
        path_name=r"../SE_test/" + path_name + ".fif"     
    path_name=path_name.replace('/','\\')
    print(path_name)
    raw_temp.save(path_name,overwrite=True)

In [38]:
file_without_se = []
df_parameter_train=pd.DataFrame(columns=['parameter','result'])
df_parameter_test=pd.DataFrame(columns=['parameter','result'])

In [39]:
#训练集SE切片长度
train_SE_len=16
#测试集SE切片长度
test_SE_len=16
#选择offset是多少的倍数
# offset_multiple=2

In [40]:
#生成SE_train
for file_path in train_therapy_file_path:
    if judgeflag(file_path,"SE"):
        #初始化random_flip_no，random_flip_no的作用是方便命名未发作文件，防止发生同名文件覆盖
        random_flip_no=0
        #如果发作，则只截取train_SE_len长度
        #为False时表示当前数据送入训练集
        #通过8种策略数据增强
        for DA_strategy in range(5):
            get_se_epochs(file_path,train_SE_len,False,DA_strategy,0)
            #添加参数到df_parameter_train的末尾
            df_parameter_train.loc[len(df_parameter_train.index)]=[path_parameter_dict[file_path],path_result_dict[file_path]]
            #如果未发作，则随机通道翻转增加未发作样本数量
            if path_result_dict[file_path]==0:
                for j in range(1,multiple):
                    #修改random_flip_no
                    random_flip_no+=1
                    #DA_strategy固定为8，即通过随机翻转达到0-1平衡
                    get_se_epochs(file_path,train_SE_len,False,8,random_flip_no) 
                    df_parameter_train.loc[len(df_parameter_train.index)]=[path_parameter_dict[file_path],path_result_dict[file_path]]
    else:
        file_without_se.append(file_path)
#生成SE_test,不考虑对未发作数据的重叠采样
for file_path in test_therapy_file_path:
    if judgeflag(file_path,"SE"):
        #取test_SE_len片段
        #DA_strategy=0,即不做变化
        get_se_epochs(file_path,test_SE_len,True,0,0) 
        df_parameter_test.loc[len(df_parameter_test.index)]=[path_parameter_dict[file_path],path_result_dict[file_path]]
    else:
        file_without_se.append(file_path)

Extracting EDF parameters from E:\jupyter file\EEGprocess\patient_data\治疗脑电-导出\第四批次治疗脑电\平均参考\AV-严聪慧-20220803.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['Recording ends', 'Recording starts', 'SE', 'end', 'start']
Extracting EDF parameters from E:\jupyter file\EEGprocess\patient_data\治疗脑电-导出\第四批次治疗脑电\平均参考\AV-严聪慧-20220803.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
['EEG Fp1-AV', 'EEG Fp2-AV', 'EEG F3-AV', 'EEG F4-AV', 'EEG C3-AV', 'EEG C4-AV', 'EEG P3-AV', 'EEG P4-AV', 'EEG O1-AV', 'EEG O2-AV', 'EEG F7-AV', 'EEG F8-AV', 'EEG T3-AV', 'EEG T4-AV', 'EEG T5-AV', 'EEG T6-AV', 'EEG Fz-AV', 'EEG Cz-AV', 'EEG Pz-AV']
Used Annotations descriptions: ['Recording ends', 'Recording starts', 'SE', 'end', 'start']
1774 9966
[ 3.46484375  3.46679688  3.46875    ... 19.45898438 19.4609375
 19.46289062]
Creating RawArray with float64 data, n_channels=15, n_times=8192
    Range : 0 .

C:\Users\Administrator\AppData\Local\Temp\ipykernel_1316\2263234904.py:75: RuntimeWarning: This filename (E:\jupyter file\EEGprocess\work_file\..\SE_train\AV-严聪慧-20220803(original).fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_temp.save(path_name,overwrite=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_1316\2263234904.py:75: RuntimeWarning: This filename (E:\jupyter file\EEGprocess\work_file\..\SE_train\AV-严聪慧-20220803random_flip_1.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_temp.save(path_name,overwrite=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_1316\2263234904.py:75: RuntimeWarni

Creating RawArray with float64 data, n_channels=15, n_times=8192
    Range : 0 ... 8191 =      0.000 ...    15.998 secs
Ready.
..\SE_train\AV-严聪慧-20220803(jitter).fif
Overwriting existing file.
Writing E:\jupyter file\EEGprocess\work_file\..\SE_train\AV-严聪慧-20220803(jitter).fif
Closing E:\jupyter file\EEGprocess\work_file\..\SE_train\AV-严聪慧-20220803(jitter).fif
[done]
Extracting EDF parameters from E:\jupyter file\EEGprocess\patient_data\治疗脑电-导出\第四批次治疗脑电\平均参考\AV-严聪慧-20220803.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
['EEG Fp1-AV', 'EEG Fp2-AV', 'EEG F3-AV', 'EEG F4-AV', 'EEG C3-AV', 'EEG C4-AV', 'EEG P3-AV', 'EEG P4-AV', 'EEG O1-AV', 'EEG O2-AV', 'EEG F7-AV', 'EEG F8-AV', 'EEG T3-AV', 'EEG T4-AV', 'EEG T5-AV', 'EEG T6-AV', 'EEG Fz-AV', 'EEG Cz-AV', 'EEG Pz-AV']
Used Annotations descriptions: ['Recording ends', 'Recording starts', 'SE', 'end', 'start']
1774 9966
[ 3.46484375  3.46679688  3.46875    ... 19.45898438 19.4609375
 19.46289062]


Ready.
..\SE_train\AV-严聪慧-20220803random_flip_11.fif
Overwriting existing file.
Writing E:\jupyter file\EEGprocess\work_file\..\SE_train\AV-严聪慧-20220803random_flip_11.fif
Closing E:\jupyter file\EEGprocess\work_file\..\SE_train\AV-严聪慧-20220803random_flip_11.fif
[done]
Extracting EDF parameters from E:\jupyter file\EEGprocess\patient_data\治疗脑电-导出\第四批次治疗脑电\平均参考\AV-严聪慧-20220803.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
['EEG Fp1-AV', 'EEG Fp2-AV', 'EEG F3-AV', 'EEG F4-AV', 'EEG C3-AV', 'EEG C4-AV', 'EEG P3-AV', 'EEG P4-AV', 'EEG O1-AV', 'EEG O2-AV', 'EEG F7-AV', 'EEG F8-AV', 'EEG T3-AV', 'EEG T4-AV', 'EEG T5-AV', 'EEG T6-AV', 'EEG Fz-AV', 'EEG Cz-AV', 'EEG Pz-AV']
Used Annotations descriptions: ['Recording ends', 'Recording starts', 'SE', 'end', 'start']
1774 9966
[ 3.46484375  3.46679688  3.46875    ... 19.45898438 19.4609375
 19.46289062]
Creating RawArray with float64 data, n_channels=15, n_times=8192
    Range : 0 ... 8191 =      0.000 .

C:\Users\Administrator\AppData\Local\Temp\ipykernel_1316\2263234904.py:75: RuntimeWarning: This filename (E:\jupyter file\EEGprocess\work_file\..\SE_train\AV-严聪慧-20220803(jitter).fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_temp.save(path_name,overwrite=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_1316\2263234904.py:75: RuntimeWarning: This filename (E:\jupyter file\EEGprocess\work_file\..\SE_train\AV-严聪慧-20220803random_flip_5.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_temp.save(path_name,overwrite=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_1316\2263234904.py:75: RuntimeWarning

...

EDF file detected
Setting channel info structure...
Creating raw.info structure...
['EEG Fp1-AV', 'EEG Fp2-AV', 'EEG F3-AV', 'EEG F4-AV', 'EEG C3-AV', 'EEG C4-AV', 'EEG P3-AV', 'EEG P4-AV', 'EEG O1-AV', 'EEG O2-AV', 'EEG F7-AV', 'EEG F8-AV', 'EEG T3-AV', 'EEG T4-AV', 'EEG T5-AV', 'EEG T6-AV', 'EEG Fz-AV', 'EEG Cz-AV', 'EEG Pz-AV']
Used Annotations descriptions: ['Recording ends', 'Recording starts', 'SE', 'end', 'start']
14804 22996
[28.9140625  28.91601562 28.91796875 ... 44.90820312 44.91015625
 44.91210938]
Creating RawArray with float64 data, n_channels=15, n_times=8192
    Range : 0 ... 8191 =      0.000 ...    15.998 secs
Ready.
..\SE_test\AV-成思思-20220803.fif
Overwriting existing file.
Writing E:\jupyter file\EEGprocess\work_file\..\SE_test\AV-成思思-20220803.fif
Closing E:\jupyter file\EEGprocess\work_file\..\SE_test\AV-成思思-20220803.fif
[done]
Extracting EDF parameters from E:\jupyter file\EEGprocess\patient_data\治疗脑电-导出\第四批次治疗脑电\平均参考\AV-成思思-20220805.edf...
EDF file detected
Settin

C:\Users\Administrator\AppData\Local\Temp\ipykernel_1316\2263234904.py:75: RuntimeWarning: This filename (E:\jupyter file\EEGprocess\work_file\..\SE_test\AV-吴妍-20220819.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_temp.save(path_name,overwrite=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_1316\2263234904.py:75: RuntimeWarning: This filename (E:\jupyter file\EEGprocess\work_file\..\SE_test\AV-吴妍-20220822.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_temp.save(path_name,overwrite=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_1316\2263234904.py:75: RuntimeWarning: This filename (E:\jupyt

Used Annotations descriptions: ['Recording ends', 'Recording starts', 'SE', 'end', 'start']
Extracting EDF parameters from E:\jupyter file\EEGprocess\patient_data\治疗脑电-导出\第四批次治疗脑电\平均参考\AV-成思思-20220805.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
['EEG Fp1-AV', 'EEG Fp2-AV', 'EEG F3-AV', 'EEG F4-AV', 'EEG C3-AV', 'EEG C4-AV', 'EEG P3-AV', 'EEG P4-AV', 'EEG O1-AV', 'EEG O2-AV', 'EEG F7-AV', 'EEG F8-AV', 'EEG T3-AV', 'EEG T4-AV', 'EEG T5-AV', 'EEG T6-AV', 'EEG Fz-AV', 'EEG Cz-AV', 'EEG Pz-AV']
Used Annotations descriptions: ['Recording ends', 'Recording starts', 'SE', 'end', 'start']
16505 24697
[32.23632812 32.23828125 32.24023438 ... 48.23046875 48.23242188
 48.234375  ]
Creating RawArray with float64 data, n_channels=15, n_times=8192
    Range : 0 ... 8191 =      0.000 ...    15.998 secs
Ready.
..\SE_test\AV-成思思-20220805.fif
Overwriting existing file.
Writing E:\jupyter file\EEGprocess\work_file\..\SE_test\AV-成思思-20220805.fif
Closing E:\jupy

19159 27351
[37.41992188 37.421875   37.42382812 ... 53.4140625  53.41601562
 53.41796875]
Creating RawArray with float64 data, n_channels=15, n_times=8192
    Range : 0 ... 8191 =      0.000 ...    15.998 secs
Ready.
..\SE_test\AV-罗文彤-20220727.fif
Overwriting existing file.
Writing E:\jupyter file\EEGprocess\work_file\..\SE_test\AV-罗文彤-20220727.fif
Closing E:\jupyter file\EEGprocess\work_file\..\SE_test\AV-罗文彤-20220727.fif
[done]


C:\Users\Administrator\AppData\Local\Temp\ipykernel_1316\2263234904.py:75: RuntimeWarning: This filename (E:\jupyter file\EEGprocess\work_file\..\SE_test\AV-成思思-20220805.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_temp.save(path_name,overwrite=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_1316\2263234904.py:75: RuntimeWarning: This filename (E:\jupyter file\EEGprocess\work_file\..\SE_test\AV-成思思-20220808.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_temp.save(path_name,overwrite=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_1316\2263234904.py:75: RuntimeWarning: This filename (E:\jup

In [41]:
file_without_se

[]

In [42]:
df_parameter_train

,parameter,result
0,4,0
1,4,0
2,4,0
3,4,0
4,4,0
...,...,...
670,1,1
671,1,1
672,1,1
673,1,1


In [43]:
df_parameter_test

,parameter,result
0,2,1
1,3,1
2,4,1
3,1,1
4,1,1
5,4,1
6,2,0
7,3,1
8,2,1
9,4,1


In [44]:
train_output_path=("../SE_parameter/train_matched_stimulate_parameter.xlsx")
test_output_path=("../SE_parameter/test_matched_stimulate_parameter.xlsx")

In [45]:
df_parameter_train.to_excel(train_output_path,sheet_name="Sheet1")

In [46]:
df_parameter_test.to_excel(test_output_path,sheet_name="Sheet1")